This first cell sets up my environment so the rest of the notebook runs smoothly and consistently. I start by installing specific versions of plotly, pandas, and openpyxl to avoid any compatibility issues and to ensure that my visualizations and data operations behave the same way every time the notebook is run. Then I import pandas for data manipulation and plotly.express for building interactive plots. Finally, I set pio.renderers.default = "colab" so that all Plotly charts automatically display inside Google Colab without needing any extra configuration.

In [ ]:

!pip install plotly==5.24.0 pandas==2.2.2 openpyxl==3.1.5 -q

import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "colab"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 96.0 MB/s eta 0:00:00


Below cell allows me to upload the Alzheimer dataset directly into Google Colab instead of relying on a local file path. I use files.upload() to open a file-picker so I can select the Excel file from my computer. Once the upload is complete, Colab stores the file temporarily, and I read it using pd.read_excel() with the correct sheet name. This makes the workflow flexible because I can reuse the notebook on any machine without changing paths. I also print the shape of the dataset and display the first few rows to confirm that the file loaded correctly and the structure looks as expected.

In [ ]:

from google.colab import files

uploaded = files.upload()

file_name = 'DAAN_871_ALZHEIMERS_NPA_CLEANED.xlsx'
df = pd.read_excel(file_name, sheet_name='Sheet1')

print("Data shape:", df.shape)
df.head()

Saving DAAN_871_ALZHEIMERS_NPA_CLEANED.xlsx to DAAN_871_ALZHEIMERS_NPA_CLEANED.xlsx
Data shape: (202499, 10)


,Product Sum,Brand/Generic,Gender,Age,Month,Physician Specialty,Sales Super Channel,Sales Channel,TRx,TRx Pharmacy $
0,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2023-04-01,NEUROLOGY,RETAIL,COMBINED RETAIL,1,542
1,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2023-11-01,NEUROLOGY,RETAIL,COMBINED RETAIL,1,577
2,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2024-04-01,NURSE PRACTITIONER,RETAIL,COMBINED RETAIL,1,582
3,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2024-05-01,NURSE PRACTITIONER,RETAIL,COMBINED RETAIL,1,578
4,ADLARITY,BRANDED GENERIC,FEMALE,40 TO 59,2022-10-01,INTERNAL MEDICINE,NON-RETAIL,LONG-TERM CARE,1,323


Below cell cleans and formats the dataset so it’s ready for plotting and animation. First, I convert the Month column to a proper datetime type so that time-based operations and plots work correctly. Then I create a new YearMonth column, which stores the month in YYYY-MM format; this is especially useful for cleaner axis labels and for driving animations over time. Next, I explicitly convert the TRx and TRx Pharmacy $ columns to numeric, coercing any problematic values to NaN so they don’t break calculations or charts. Finally, I print a subset of key columns to quickly verify that the transformations look correct and that the core demographic, prescriber, channel, and metric fields are all present and properly formatted.

In [ ]:

df['Month'] = pd.to_datetime(df['Month'])

df['YearMonth'] = df['Month'].dt.to_period('M').astype(str)

df['TRx'] = pd.to_numeric(df['TRx'], errors='coerce')
df['TRx Pharmacy $'] = pd.to_numeric(df['TRx Pharmacy $'], errors='coerce')

df[['Product Sum', 'Brand/Generic', 'Gender', 'Age', 'Month', 'YearMonth',
    'Physician Specialty', 'Sales Super Channel', 'Sales Channel',
    'TRx', 'TRx Pharmacy $']].head()

,Product Sum,Brand/Generic,Gender,Age,Month,YearMonth,Physician Specialty,Sales Super Channel,Sales Channel,TRx,TRx Pharmacy $
0,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2023-04-01,2023-04,NEUROLOGY,RETAIL,COMBINED RETAIL,1,542
1,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2023-11-01,2023-11,NEUROLOGY,RETAIL,COMBINED RETAIL,1,577
2,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2024-04-01,2024-04,NURSE PRACTITIONER,RETAIL,COMBINED RETAIL,1,582
3,ADLARITY,BRANDED GENERIC,FEMALE,20 TO 39,2024-05-01,2024-05,NURSE PRACTITIONER,RETAIL,COMBINED RETAIL,1,578
4,ADLARITY,BRANDED GENERIC,FEMALE,40 TO 59,2022-10-01,2022-10,INTERNAL MEDICINE,NON-RETAIL,LONG-TERM CARE,1,323


Below cell creates a sophisticated animated time-series display that illustrates the monthly evolution of each Alzheimer product's Total TRx, enabling me to see the trend both forward and backward. To make sure the animation shows in the correct chronological order, I first sort the dataset by month. In order to create the animation frames dynamically, I then extract the distinct list of months and products.

I manually create a dictionary that associates each (Month, Product) pair with its matching TRx value in order to prevent column-indexing problems. This avoids missing-column naming issues and guarantees clean lookups. I plot complete lines for every product throughout the whole date range for the first display, but animation frames will subsequently be used to determine how these lines move.

Forward playback: As TRx data builds up, lines progressively show up each month.
In reverse playback, the chart gradually removes months as it rewinds backward.

I suppress future values with None and just show the part of the TRx line that is present up to that month for each frame. Instead of displaying the entire history at once, this enables the animation to "draw" the time series smoothly.

After that, I create a custom slider that lets the user manually scroll between months to see how TRx has changed. Play and Reverse are two interactive buttons in the layout that provide complete control over the animation's path. In order to properly evaluate each product's journey throughout the entire timeline, the picture is finally presented with appropriate axis headings and a single legend.

**Business Question 1**

**How have Total TRx volumes for key Alzheimer’s therapies changed over time, and which brands are gaining or losing market momentum relative to competitors?**

In [25]:
import plotly.graph_objects as go
import numpy as np

month_time = month_time.sort_values("Month")


months_unique = month_time["Month"].sort_values().unique()
products_unique = month_time["Product Sum"].unique()


data_map = {}
for _, r in month_time.iterrows():
    data_map[(r["Month"], r["Product Sum"])] = r["TRx"]


initial_month = months_unique[0]

fig1 = go.Figure()

for prod in products_unique:
    y_vals = []
    for m in months_unique:
        y_vals.append(data_map.get((m, prod), 0))

    fig1.add_trace(
        go.Scatter(
            x=months_unique,
            y=y_vals,
            mode="lines+markers",
            name=str(prod),
            visible=True
        )
    )

frames_forward = []
for i, m in enumerate(months_unique):
    frame_data = []
    for t_idx, prod in enumerate(products_unique):

        y_vals = []
        for j in range(i + 1):
            y_vals.append(data_map.get((months_unique[j], prod), 0))

        y_frame = y_vals + [None] * (len(months_unique) - len(y_vals))
        frame_data.append(
            go.Scatter(
                x=months_unique,
                y=y_frame,
                mode="lines+markers",
                name=str(prod)
            )
        )
    frames_forward.append(go.Frame(data=frame_data, name=str(m)))


frames_reverse = []
for i, m in enumerate(months_unique[::-1]):
    frame_data = []
    for t_idx, prod in enumerate(products_unique):

        y_vals = []
        for j in range(i + 1):
            y_vals.append(data_map.get((months_unique[::-1][j], prod), 0))

        y_frame = y_vals + [None] * (len(months_unique) - len(y_vals))
        frame_data.append(
            go.Scatter(
                x=months_unique[::-1],
                y=y_frame,
                mode="lines+markers",
                name=str(prod)
            )
        )
    frames_reverse.append(go.Frame(data=frame_data, name="rev_" + str(m)))


fig1.frames = frames_forward + frames_reverse


slider_steps = []
for i, m in enumerate(months_unique):
    slider_steps.append(
        dict(
            method="animate",
            args=[
                [
                    str(m)
                ],
                dict(frame=dict(duration=400, redraw=True),
                     mode="immediate",
                     transition=dict(duration=0))
            ],
            label=str(m)[:10]
        )
    )


fig1.update_layout(
    title="Total TRx Over Time by Product",
    xaxis_title="Month",
    yaxis_title="Total TRx",
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.1,
            y=1.15,
            pad=dict(r=10, t=10),
            showactive=False,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        [
                            str(m) for m in months_unique
                        ],
                        dict(
                            frame=dict(duration=500, redraw=True),
                            fromcurrent=True,
                            transition=dict(duration=0)
                        )
                    ]
                ),
                dict(
                    label="Reverse",
                    method="animate",
                    args=[
                        [
                            "rev_" + str(m) for m in months_unique[::-1]
                        ],
                        dict(
                            frame=dict(duration=500, redraw=True),
                            fromcurrent=True,
                            transition=dict(duration=0)
                        )
                    ]
                ),
            ]
        )
    ],
    sliders=[
        dict(
            active=0,
            currentvalue=dict(prefix="Month: "),
            pad=dict(t=50),
            steps=slider_steps
        )
    ]
)

fig1.show()

I begin by rearranging the data so that it is at the Month × Physician Specialty level. I arrange the primary df by ['Month', 'Physician Specialty'] and sum TRx to determine the total number of prescriptions written by each specialty in each month. I create a string version of Month_str (YYYY-MM) to use as the animation's frame title after making sure Month is in the proper datetime format.

I identify the top 15 specializations by total TRx for the entire time period to keep the display understandable and focused. I calculate the total TRx by specialization, sort in descending order, select the top N, and then narrow the dataset down to just those specialties. This indicates that the animation focuses on the prescriber groups that are most commercially relevant.

I then determine the monthly percentage share of TRx for each specialty. I calculate the overall TRx for each Month_str and divide the TRx for each specialty by the monthly total to obtain the TRx_share in percentage. I now have two complementing metrics:

Absolute volume, or TRx
TRx_share (the monthly market's relative contribution)

To ensure that the animation plays in the proper chronological order, order the months and convert Month_str to an ordered categorical.

I use Plotly Go to create a horizontal bar chart for the display.Figure. Specialties are neatly stacked from lowest to highest on the y-axis in the first frame, which uses the first month's data sorted by TRx. Two bar traces are added by me:

By default, Trace 0-TRx (Absolute), which displays the quantity of prescriptions for each specialty, is visible.

The percentage share for each specialty is contained in Trace 1-TRx Share (%), which was formerly buried.

After that, I create a set of animation frames, one every month. I maintain a constant specialty order by updating both traces in each frame: the TRx bars and the TRx_share bars for that particular month. These frames are used by Plotly to animate how the bars change over time.

Lastly, I add three crucial interaction components to personalize the layout:

I have a Play/Pause button that allows me to observe how the prescriber mix changes on a monthly basis.

I can scrub the time slider at the bottom to go straight to any point in time, with each step being a month.

Toggle the x-axis between "TRx (Absolute)" and "% TRx Share" using a dropdown menu. This allows me to quickly change the focus from sheer volume to relative importance without having to redo the chart.

**Business Question 2:**

**Which physician specialties are driving growth in Alzheimer prescriptions over time, and how is each specialty’s share of total TRx shifting month by month?**

In [28]:
import pandas as pd
import plotly.graph_objects as go


spec_month_anim = df.groupby(['Month', 'Physician Specialty'])['TRx'].sum().reset_index()

if not pd.api.types.is_datetime64_any_dtype(spec_month_anim["Month"]):
    spec_month_anim["Month"] = pd.to_datetime(spec_month_anim["Month"])

spec_month_anim["Month_str"] = spec_month_anim["Month"].dt.strftime("%Y-%m")

top_n = 15
top_specs = (
    spec_month_anim.groupby("Physician Specialty")["TRx"]
    .sum()
    .sort_values(ascending=False)
    .head(top_n)
    .index
)
spec_month_anim = spec_month_anim[
    spec_month_anim["Physician Specialty"].isin(top_specs)
]

month_totals = spec_month_anim.groupby("Month_str")["TRx"].transform("sum")
spec_month_anim["TRx_share"] = spec_month_anim["TRx"] / month_totals * 100

months_order = sorted(spec_month_anim["Month_str"].unique())
spec_month_anim["Month_str"] = pd.Categorical(
    spec_month_anim["Month_str"],
    categories=months_order,
    ordered=True,
)

first_month = months_order[0]
df0 = spec_month_anim[spec_month_anim["Month_str"] == first_month].copy()
df0 = df0.sort_values("TRx", ascending=True)

fig = go.Figure()


fig.add_trace(
    go.Bar(
        x=df0["TRx"],
        y=df0["Physician Specialty"],
        orientation="h",
        name="TRx",
        marker=dict(color="steelblue"),
        visible=True,
    )
)

fig.add_trace(
    go.Bar(
        x=df0["TRx_share"],
        y=df0["Physician Specialty"],
        orientation="h",
        name="TRx Share (%)",
        marker=dict(color="seagreen"),
        visible=False,
    )
)


frames = []
for m in months_order:
    dfm = spec_month_anim[spec_month_anim["Month_str"] == m].copy()
    dfm = dfm.sort_values("TRx", ascending=True)

    frame = go.Frame(
        name=m,
        data=[

            go.Bar(
                x=dfm["TRx"],
                y=dfm["Physician Specialty"],
                orientation="h",
            ),

            go.Bar(
                x=dfm["TRx_share"],
                y=dfm["Physician Specialty"],
                orientation="h",
            ),
        ],
    )
    frames.append(frame)

fig.frames = frames


fig.update_layout(
    title="Prescriber Mix by Specialty Over Time",
    xaxis_title="TRx",
    yaxis_title="Physician Specialty",
    yaxis=dict(categoryorder="array", categoryarray=df0["Physician Specialty"]),
    updatemenus=[

        dict(
            type="buttons",
            direction="left",
            x=0.1,
            y=-0.1,
            showactive=False,
            buttons=[
                dict(
                    label="Play",
                    method="animate",
                    args=[
                        None,
                        {
                            "frame": {"duration": 400, "redraw": True},
                            "transition": {"duration": 200},
                            "fromcurrent": True,
                        },
                    ],
                ),
                dict(
                    label="Pause",
                    method="animate",
                    args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}],
                ),
            ],
        ),

        dict(
            type="dropdown",
            direction="down",
            x=0.0,
            y=1.15,
            showactive=True,
            buttons=[
                dict(
                    label="TRx (Absolute)",
                    method="update",
                    args=[
                        {"visible": [True, False]},
                        {"xaxis": {"title": "TRx"}},
                    ],
                ),
                dict(
                    label="% TRx Share",
                    method="update",
                    args=[
                        {"visible": [False, True]},
                        {"xaxis": {"title": "TRx Share (%)"}},
                    ],
                ),
            ],
        ),
    ],
    sliders=[
        dict(
            active=0,
            y=-0.18,
            x=0.1,
            len=0.9,
            pad=dict(t=50),
            steps=[
                dict(
                    label=m,
                    method="animate",
                    args=[
                        [m],
                        {
                            "mode": "immediate",
                            "frame": {"duration": 0, "redraw": True},
                            "transition": {"duration": 0},
                        },
                    ],
                )
                for m in months_order
            ],
        )
    ],
)

fig.show()

The physician specialties that produce the highest TRx across various sales channels are displayed in this heatmap. I total TRx and TRx Pharmacy $. I group the data by Physician Specialty and Sales Channel. These data are then shown by Plotly as color intensity; higher prescription volume is indicated by darker cells. This provides a quick visual representation of the concentration of prescribed strength and the most important specialty-channel combinations.

This region chart shows the monthly variations in each sales channel's percentage of TRx. I use groupnorm ("fraction") to normalize each month after aggregating TRx at the Month × Sales Channel level, so the figure shows percentage contribution instead of raw numbers. This makes it simple to determine whether a certain channel's share is increasing, decreasing, or staying constant over time.


**Business Question 4:**

**Which specialty channel combinations drive the highest TRx, and where are there untapped channel opportunities for key specialties?**


**Business Question 5:**

**How is the TRx channel mix evolving over time, and which channels are gaining or losing share in the Alzheimer market?**

In [31]:
import plotly.express as px


heat_agg = df.groupby(
    ['Physician Specialty', 'Sales Channel'],
    as_index=False
)[['TRx', 'TRx Pharmacy $']].sum()

fig_heat = px.density_heatmap(
    heat_agg,
    x="Sales Channel",
    y="Physician Specialty",
    z="TRx",
    color_continuous_scale="Viridis",
    hover_data=["TRx Pharmacy $"],
    title="TRx by Physician Specialty and Sales Channel"
)

fig_heat.update_layout(
    xaxis_title="Sales Channel",
    yaxis_title="Physician Specialty"
)

fig_heat.show()


channel_time = df.groupby(['Month', 'Sales Channel'], as_index=False)[['TRx']].sum()

fig_channel = px.area(
    channel_time,
    x="Month",
    y="TRx",
    color="Sales Channel",
    title="Channel Mix of TRx Over Time",
    groupnorm="fraction"
)

fig_channel.update_layout(
    xaxis_title="Month",
    yaxis_title="Share of TRx (if groupnorm='fraction')",
    legend_title="Sales Channel",
    hovermode="x unified"
)

fig_channel.show()
